In [3]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import numpy as np
import pandas as pd
import pathway as pw
import math
from bokeh.plotting import figure,show
from datetime import datetime,timedelta
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.models import Range1d, LinearAxis
output_notebook()


# Constants
BASE_PRICE = 10              # Base price in dollars
TIME_POINTS_PER_DAY = 18     # From 8:00 AM to 4:30 PM with 30-minute intervals

MODEL-1

In [5]:
def model1_baseline_linear(previous_price, occupancy, capacity, alpha=0.5):
    """
    Simple linear model where price increases with occupancy rate

    Args:
        previous_price: Price at previous time step
        occupancy: Current number of parked vehicles
        capacity: Maximum capacity of the parking lot
        alpha: Sensitivity parameter (default 0.5)

    Returns:
        New price for the parking lot
    """
    occupancy_rate = occupancy / capacity
    new_price = previous_price + alpha * occupancy_rate
    return max(BASE_PRICE * 0.5, min(BASE_PRICE * 2, new_price))  # Bound the price 50% to 200% of base_price

MODEL-2

In [6]:
def model2_demand_based(previous_price, current_state, params):
    """
    Advanced demand-based pricing model

    Args:
        previous_price: Price at previous time step
        current_state: Dictionary containing:
            - occupancy: Current number of parked vehicles
            - capacity: Maximum capacity
            - queue_length: Vehicles waiting to enter
            - traffic_level: Nearby traffic congestion (0-1)
            - is_special_day: Boolean for special events/holidays
            - vehicle_type: Type of incoming vehicle (car, bike, truck)
        params: Dictionary of model parameters (alpha, beta, gamma, delta, epsilon, lambda)

    Returns:
        New price for the parking lot
    """
    # Extract features
    occupancy = current_state['occupancy']
    capacity = current_state['capacity']
    queue_length = current_state['queue_length']
    traffic_level = current_state['traffic_level']
    is_special_day = current_state['is_special_day']
    vehicle_type = current_state['vehicle_type']

    # Vehicle type weights (trucks pay more, bikes pay less)
    vehicle_weights = {'car': 1.0, 'truck': 1.5, 'bike': 0.7}
    vehicle_weight = vehicle_weights.get(vehicle_type, 1.0)

    # Calculate demand components
    occupancy_component = params['alpha'] * (occupancy / capacity)
    queue_component = params['beta'] * (queue_length / capacity)  # Normalized queue length
    traffic_component = params['gamma'] * traffic_level
    special_day_component = params['delta'] * is_special_day
    vehicle_component = params['epsilon'] * vehicle_weight

    # Combined demand score
    demand = (occupancy_component + queue_component - traffic_component +
              special_day_component + vehicle_component)

    # Normalize demand to [0, 1] range
    max_possible_demand = (params['alpha'] + params['beta'] + params['delta'] +
                          params['epsilon'] * 1.5)  # Max vehicle weight is 1.5 for trucks
    normalized_demand = min(max(demand / max_possible_demand, 0), 1)

    # Calculate new price
    new_price = BASE_PRICE * (1 + params['lambda'] * normalized_demand)

    # Ensure price is within bounds and changes smoothly
    max_change = BASE_PRICE * 0.1  # Limit price change to 10% of base price per step
    bounded_price = max(BASE_PRICE * 0.5, min(BASE_PRICE * 2, new_price))
    smoothed_price = previous_price + max(-max_change, min(max_change, bounded_price - previous_price))

    return smoothed_price

# Example parameters for Model 2
model2_params = {
    'alpha': 0.6,   # Occupancy weight
    'beta': 0.4,    # Queue length weight
    'gamma': 0.3,   # Traffic weight (negative impact)
    'delta': 0.5,   # Special day weight
    'epsilon': 0.2, # Vehicle type weight
    'lambda': 1.0   # Demand multiplier
}

MODEL-3

In [7]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points
    on the Earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c*r

def model3_competitive(previous_price, current_state, competitors, params):
    """
    Competitive pricing model that considers nearby parking lots

    Args:
        previous_price: Price at previous time step
        current_state: Dictionary containing parking lot state
        competitors: List of dictionaries with competitor info
        params: Dictionary of model parameters

    Returns:
        New price and optionally a rerouting suggestion
    """
    # First calculate demand-based price (from Model 2)
    demand_price = model2_demand_based(previous_price, current_state, params)

    # Find nearby competitors (within 1km)
    nearby_competitors = [
        comp for comp in competitors
        if haversine_distance(
            current_state['latitude'], current_state['longitude'],
            comp['latitude'], comp['longitude']
        ) <= 1.0
    ]

    if not nearby_competitors:
        return demand_price, "No nearby competitors"

    # Calculate competitive metrics
    competitor_prices = [c['price'] for c in nearby_competitors]
    avg_competitor_price = np.mean(competitor_prices)
    min_competitor_price = min(competitor_prices)
    max_competitor_price = max(competitor_prices)

    # Calculate competitor availability
    competitor_availability = [
        (c['capacity'] - c['occupancy']) / c['capacity']
        for c in nearby_competitors
    ]
    avg_competitor_availability = np.mean(competitor_availability)

    # Calculate base competitive price using weighted average
    base_competitive_price = (demand_price * (1 - params['competitive_weight']) + avg_competitor_price * params['competitive_weight'])

    # Adjust price based on market conditions
    if current_state['occupancy'] / current_state['capacity'] > 0.9:  # Nearly full
        if demand_price > min_competitor_price and avg_competitor_availability > 0.2:
            # We're more expensive and competitors have space - be competitive
            competitive_price = min(
                base_competitive_price,
                min_competitor_price * 1.1  # Don't undercut too much
            )
            suggestion = f"Suggest rerouting to {len([p for p in nearby_competitors if p['price'] < demand_price])} cheaper alternatives"
        else:
            # No good alternatives - keep price high but consider max competitor price
            competitive_price = min(
                base_competitive_price,
                max_competitor_price * 1.05  # Don't exceed market max too much
            )
            suggestion = "No rerouting - competitors also busy"
    else:
        # We have space - price competitively within market bounds
        competitive_price = np.clip(
            base_competitive_price,
            min_competitor_price * 0.9,  # Can undercut slightly
            max_competitor_price * 1.1   # Can premium slightly
        )
        suggestion = "Pricing within competitive market range"

    # Ensure absolute price bounds are respected
    final_price = max(BASE_PRICE * 0.5, min(BASE_PRICE * 2, competitive_price))

    return final_price, suggestion

# Extended parameters for Model 3 (includes Model 2 params plus competitive weights)
model3_params = {
    **model2_params,  # unpack dictionary(**) - Include all Model 2 parameters
    'competitive_weight': 0.3  # How much to weight competitor prices
}

In [8]:
# Load your dataset with all columns to google colab
df = pd.read_csv('dataset.csv')
# Convert date and time to a single timestamp
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Map traffic conditions to numerical values
traffic_mapping = {'low': 0.3, 'medium': 0.6, 'high': 0.9}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_mapping)

# Ensure QueueLength is numeric and fill missing values
df['QueueLength'] = pd.to_numeric(df['QueueLength'], errors='coerce').fillna(0)

# Create vehicle type weights
df['VehicleWeight'] = df['VehicleType'].map({'car': 1.0, 'truck': 1.5, 'bike': 0.7})

# Sort by timestamp to ensure chronological processing
df = df.sort_values(['SystemCodeNumber', 'timestamp'])
print(df.head())

   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2      BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3      BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4      BHMBCCMKT01       577  26.144536  91.736172        150        bike   

  TrafficConditionNearby  QueueLength  IsSpecialDay LastUpdatedDate  \
0                    low            1             0      04-10-2016   
1                    low            1             0      04-10-2016   
2                    low            2             0      04-10-2016   
3                    low            2             0      04-10-2016   
4                    low            2             0      04-10-2016   

  LastUpdatedTime           timestamp  TrafficLevel  VehicleWeight  
0        07:59:00

In [9]:
def simulate_pricing_for_visualization(df, system_code, days_to_show=3):
    """
    Simulate pricing for a specific parking lot over time using all three models
    Returns a DataFrame with timestamps and prices from each model
    """
    # Filter data for the selected parking lot
    lot_data = df[df['SystemCodeNumber'] == system_code].copy()

    # Sort by timestamp
    lot_data = lot_data.sort_values('timestamp')

    # Initialize prices
    price_model1 = BASE_PRICE
    price_model2 = BASE_PRICE
    price_model3 = BASE_PRICE

    # Prepare to store results
    results = []

    # Simulate each time point
    for idx, row in lot_data.iterrows():
        # Model 1 - Basic linear
        price_model1 = model1_baseline_linear(price_model1, row['Occupancy'], row['Capacity'])

        # Model 2 - Demand-based
        current_state = {
            'occupancy': row['Occupancy'],
            'capacity': row['Capacity'],
            'queue_length': row.get('QueueLength', 0),  # Use get with default
            'traffic_level': row.get('TrafficLevel', 0.5),  # Default to medium traffic
            'is_special_day': row.get('IsSpecialDay', False),  # Default to normal day
            'vehicle_type': row.get('VehicleType', 'car')  # Default to car
        }
        price_model2 = model2_demand_based(price_model2, current_state, model2_params)

        # Model 3 - Competitive (we'll simulate some competitors)
        competitors = [
            {
                'latitude': row['Latitude'] + 0.005,
                'longitude': row['Longitude'] + 0.005,
                'price': BASE_PRICE * (0.8 + np.random.random() * 0.4),  # Random price 80-120% of base
                'capacity': row['Capacity'],
                'occupancy': min(row['Capacity'], row['Occupancy'] * (0.8 + np.random.random() * 0.4))
            },
            {
                'latitude': row['Latitude'] - 0.005,
                'longitude': row['Longitude'] - 0.005,
                'price': BASE_PRICE * (0.9 + np.random.random() * 0.2),  # Random price 90-110% of base
                'capacity': row['Capacity'],
                'occupancy': min(row['Capacity'], row['Occupancy'] * (0.9 + np.random.random() * 0.2))
            }
        ]

        price_model3, _ = model3_competitive(
            price_model3,
            {
                'occupancy': row['Occupancy'],
                'capacity': row['Capacity'],
                'latitude': row['Latitude'],
                'longitude': row['Longitude'],
                'queue_length': row.get('QueueLength', 0),
                'traffic_level': row.get('TrafficLevel', 0.5),
                'is_special_day': row.get('IsSpecialDay', False),
                'vehicle_type': row.get('VehicleType', 'car')
            },
            competitors,
            model3_params
        )

        results.append({
            'timestamp': row['timestamp'],
            'occupancy': row['Occupancy'],
            'capacity': row['Capacity'],
            'occupancy_rate': row['Occupancy'] / row['Capacity'],
            'model1_price': price_model1,
            'model2_price': price_model2,
            'model3_price': price_model3
        })

    results_df = pd.DataFrame(results)

    # Filter for the last N days if requested
    if days_to_show:
        cutoff = results_df['timestamp'].max() - timedelta(days=days_to_show)
        results_df = results_df[results_df['timestamp'] >= cutoff]

    return results_df

def create_pricing_visualization(results_df, title):
    """
    Create an interactive visualization of pricing models over time
    """
    # Convert to ColumnDataSource for Bokeh
    source = ColumnDataSource(results_df)

    # Create the figure
    p = figure(
        title=title,
        x_axis_label='Time',
        y_axis_label='Price ($)',
        x_axis_type='datetime',
        width=800,
        height=400,
        tools="pan,wheel_zoom,box_zoom,reset,save"
    )

    # Add lines for each model
    p.line('timestamp', 'model1_price', source=source, line_width=2, color='blue', legend_label='Basic Linear Model')
    p.line('timestamp', 'model2_price', source=source, line_width=2, color='green', legend_label='Demand-Based Model')
    p.line('timestamp', 'model3_price', source=source, line_width=2, color='red', legend_label='Competitive Model')

    # Add hover tool
    hover = HoverTool(
        tooltips=[
            ("Time", "@timestamp{%F %H:%M}"),
            ("Basic Price", "@model1_price{$0.00}"),
            ("Demand Price", "@model2_price{$0.00}"),
            ("Competitive Price", "@model3_price{$0.00}"),
            ("Occupancy", "@occupancy/@capacity (@occupancy_rate{0.0%})")
        ],
        formatters={
            '@timestamp': 'datetime'
        }
    )
    p.add_tools(hover)

    # Add secondary axis for occupancy
    p.extra_y_ranges = {"occupancy": Range1d(start=0, end=1)}
    p.add_layout(LinearAxis(y_range_name="occupancy", axis_label="Occupancy Rate"), 'right')
    p.line('timestamp','occupancy_rate',source=source,line_width=1,color='gray',alpha=0.5,y_range_name="occupancy",legend_label='Occupancy Rate')

    # Styling
    p.legend.location = "bottom_right"
    p.legend.click_policy = "hide"
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_alpha = 0.5

    return p

def create_occupancy_vs_price_plot(results_df, title):
    """
    Create a scatter plot of occupancy rate vs price
    """
    source = ColumnDataSource(results_df)

    p = figure(
        title=title,
        x_axis_label='Occupancy Rate',
        y_axis_label='Price ($)',
        width=600,
        height=400,
        tools="pan,wheel_zoom,box_zoom,reset,save"
    )

    # Add scatter points for each model
    p.circle('occupancy_rate', 'model1_price', source=source, size=6, color='blue', alpha=0.6, legend_label='Basic Linear')
    p.circle('occupancy_rate', 'model2_price', source=source, size=6, color='green', alpha=0.6, legend_label='Demand-Based')
    p.circle('occupancy_rate', 'model3_price', source=source, size=6, color='red', alpha=0.6, legend_label='Competitive')

    # Add hover tool
    hover = HoverTool(
        tooltips=[
            ("Time", "@timestamp{%F %H:%M}"),
            ("Occupancy", "@occupancy_rate{0.0%}"),
            ("Basic Price", "@model1_price{$0.00}"),
            ("Demand Price", "@model2_price{$0.00}"),
            ("Competitive Price", "@model3_price{$0.00}")
        ],
        formatters={
            '@timestamp': 'datetime'
        }
    )
    p.add_tools(hover)

    # Styling
    p.legend.location = "top_left"
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_alpha = 0.5

    return p

# Example usage:
# Select a parking lot to visualize
parking_lot = "BHMBCCMKT01"  # can change this to any SystemCodeNumber from your data()

# Simulate pricing
simulation_results = simulate_pricing_for_visualization(df, parking_lot, days_to_show=3)

# Create visualizations
time_series_plot = create_pricing_visualization(
    simulation_results,
    f"Dynamic Pricing Models for Parking Lot {parking_lot} (Last 3 Days)"
)

scatter_plot = create_occupancy_vs_price_plot(
    simulation_results,
    f"Occupancy vs Price for Parking Lot {parking_lot}"
)

# Show the plots
show(gridplot([[time_series_plot], [scatter_plot]]))